In [1]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 6.8 MB/s 
time: 1.62 ms (started: 2022-12-07 10:19:37 +00:00)


# Naive Bayes

In [2]:
import pandas as pd
import numpy as np

time: 393 ms (started: 2022-12-07 10:19:37 +00:00)


In [3]:
%cd /content/drive/My Drive/Assignment-2/
df = pd.read_csv("train.csv")
df = df.drop([df.columns[0]], axis = 1)
df = df.iloc[:12000]
df

/content/drive/My Drive/Assignment-2


,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,60Nm@3500rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
1,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,60Nm@3500rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
2,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,60Nm@3500rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
3,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,113Nm@4400rpm,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,91Nm@4250rpm,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,1.132370,0.07,0.403846,C14,7788,1,B1,M8,CNG,82.1Nm@3400rpm,...,No,Yes,Yes,Yes,No,No,Yes,Yes,2,0
11996,1.132905,0.07,0.317308,C15,290,1,B1,M8,CNG,82.1Nm@3400rpm,...,No,Yes,Yes,Yes,No,No,Yes,Yes,2,0
11997,0.167369,0.03,0.548077,C10,73430,3,C2,M4,Diesel,250Nm@2750rpm,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,1
11998,0.128059,0.01,0.471154,C5,34738,1,A,M1,CNG,60Nm@3500rpm,...,No,No,No,Yes,No,No,No,Yes,0,0


time: 1.04 s (started: 2022-12-07 10:19:37 +00:00)


In [4]:
def calculate_prior(df, Y):
    classes = sorted(list(df[Y].unique()))
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

time: 1.79 ms (started: 2022-12-07 10:19:38 +00:00)


In [5]:
def calculate_likelihood_gaussian(df, feat_name, feat_val, Y, label):
    feat = list(df.columns)
    df = df[df[Y]==label]
    mean, std = df[feat_name].mean(), df[feat_name].std()
    p_x_given_y = (1 / (np.sqrt(2 * np.pi) * std)) *  np.exp(-((feat_val-mean)**2 / (2 * std**2 )))
    return p_x_given_y

time: 2.72 ms (started: 2022-12-07 10:19:38 +00:00)


In [6]:
def calculate_likelihood_categorical(df, feat_name, feat_val, Y, label):
    feat = list(df.columns)
    df = df[df[Y]==label]
    p_x_given_y = len(df[df[feat_name]==feat_val]) / len(df)
    return p_x_given_y

time: 3.4 ms (started: 2022-12-07 10:19:38 +00:00)


In [7]:
def naive_bayes(df, X, Y):
    # get feature names
    features = list(df.columns)[:-1]

    # calculate prior
    prior = calculate_prior(df, Y)

    Y_pred = []
    # loop over every data sample
    for x in X:
        # calculate likelihood
        labels = sorted(list(df[Y].unique()))
        likelihood = [1]*len(labels)
        # since first 3 columns of the sample are numerical values, the likelihood is calculated using calculate_likelihood_gaussian function for first 3 columns.
        # the likelihood of the rest columns is calculated using calculate_likelihood_categorical function 
        for j in range(len(labels)):
            for i in range(3):
                likelihood[j] *= calculate_likelihood_gaussian(df, features[i], x[i], Y, labels[j])
            for i in range(len(features)-3):
                likelihood[j] *= calculate_likelihood_categorical(df, features[i+3], x[i+3], Y, labels[j])

        # calculate posterior probability (numerator only)
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]

        Y_pred.append(np.argmax(post_prob))

    return np.array(Y_pred) 

time: 11.2 ms (started: 2022-12-07 10:19:38 +00:00)


In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=41)

X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values
Y_pred = naive_bayes(train, X=X_test, Y="is_claim")

count = 0
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_test[i]:
        count += 1
accuracy = count / len(Y_pred)
print(accuracy)

0.9179166666666667
time: 11min 37s (started: 2022-12-07 10:19:38 +00:00)


# K-Fold Cross Validation

In [10]:
kfolds_data = df.drop(columns=['is_claim'])
kfolds_target = df["is_claim"]

time: 9.92 ms (started: 2022-12-07 10:31:16 +00:00)


In [11]:
from sklearn.model_selection import KFold

time: 815 µs (started: 2022-12-07 10:44:46 +00:00)


## k = 3

In [18]:
folds_k3 = KFold(n_splits=3)

scores_nb_k3 = []

for train_index, test_index in folds_k3.split(kfolds_data, kfolds_target):
    train_k3 = df.iloc[train_index]
    X_test = kfolds_data.iloc[test_index].values
    Y_test = kfolds_target.iloc[test_index].values
    Y_pred = naive_bayes(train_k3, X=X_test, Y="is_claim")

    count = 0
    for i in range(len(Y_pred)):
        if Y_pred[i] == Y_test[i]:
            count += 1
    acc = count / len(Y_pred)
    scores_nb_k3.append(acc)

scores_nb_k3 = np.array(scores_nb_k3)
scores_nb_k3.mean()

0.8501666666666665

time: 48min 31s (started: 2022-12-07 11:03:52 +00:00)


## k = 5

In [19]:
folds_k5 = KFold(n_splits=5)

scores_nb_k5 = []

for train_index, test_index in folds_k5.split(kfolds_data, kfolds_target):
    train_k5 = df.iloc[train_index]
    X_test = kfolds_data.iloc[test_index].values
    Y_test = kfolds_target.iloc[test_index].values
    Y_pred = naive_bayes(train_k5, X=X_test, Y="is_claim")

    count = 0
    for i in range(len(Y_pred)):
        if Y_pred[i] == Y_test[i]:
            count += 1
    acc = count / len(Y_pred)
    scores_nb_k5.append(acc)

scores_nb_k5 = np.array(scores_nb_k5)
scores_nb_k5.mean()

0.85725

time: 59min 31s (started: 2022-12-07 11:54:40 +00:00)


## k = 10

In [20]:
folds_k10 = KFold(n_splits=10)

scores_nb_k10 = []

for train_index, test_index in folds_k10.split(kfolds_data, kfolds_target):
    train_k10 = df.iloc[train_index]
    X_test = kfolds_data.iloc[test_index].values
    Y_test = kfolds_target.iloc[test_index].values
    Y_pred = naive_bayes(train_k10, X=X_test, Y="is_claim")

    count = 0
    for i in range(len(Y_pred)):
        if Y_pred[i] == Y_test[i]:
            count += 1
    acc = count / len(Y_pred)
    scores_nb_k10.append(acc)

scores_nb_k10 = np.array(scores_nb_k10)
scores_nb_k10.mean()

0.8486666666666667

time: 1h 7min 9s (started: 2022-12-07 12:54:14 +00:00)
